## 스팸 문자를 Naive Bayes를 이용해 분류하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

## 1. Data

### 1.1 Data Load

- 문자 내용이 스팸인지 아닌지를 구분하기 위한 데이터.

In [4]:
spam = pd.read_csv('sms_spam.csv')

In [5]:
text = spam['text']
label = spam['type']

### 1.2 EDA

In [6]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [7]:
label[0]

'ham'

In [8]:
label.value_counts()

ham     4827
spam     747
Name: type, dtype: int64

### 1.3 Data Cleaning

- 정답의 문자를 숫자로 변환.
- ham은 0으로, spam은 1로 변환.

In [9]:
label = label.map({'ham':0, 'spam':1})

In [15]:
label.value_counts()

0    4827
1     747
Name: type, dtype: int64

- text를 문자만 존재하도록 정리.
- regex를 통해 영어, 숫자 그리고 띄어쓰기를 제외한 모든 단어를 지움.

In [10]:
re_pattern = "[^a-zA-Z0-9\ ]"

In [11]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [12]:
text.iloc[:1].str.replace(re_pattern, "", regex=True)[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [13]:
text = text.str.replace(re_pattern, "", regex=True)

- 그리고 나서 대문자들을 모두 소문자로 바꿈.

In [16]:
text[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [17]:
text.iloc[:1].str.lower()[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [18]:
text = text.str.lower()

In [19]:
text[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [20]:
text.iloc[:1].str.replace(re_pattern, "", regex=True)[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

### 1.4 Data Split

In [21]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(
    text, label, train_size=0.7, random_state=2021
)

In [22]:
print(f"train_data size: {len(train_label)}, {len(train_label)/len(text):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(text):.2f}")

train_data size: 3901, 0.70
test_data size: 1673, 0.30


## 2. Count Vectorize

- Naive Bayes를 학습시키기 위해 각 문장에서 단어들이 몇 번 나왔는지로 변환.

### 2.1 Word Tokenize

- nltk 패키지의 word_tokenize를 이용.

In [24]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jjoro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [25]:
train_text.iloc[0]

'am only searching for good dual sim mobile pa'

In [26]:
word_tokenize(train_text.iloc[0])

['am', 'only', 'searching', 'for', 'good', 'dual', 'sim', 'mobile', 'pa']

### 2.2 Count Vectorize

- 단어들을 count vector로 만듦.
- count vectorize된 matrix가 input으로 들어감.

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

예시로 2개의 문장으로 CountVectorizer를 학습.

In [28]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [29]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)

In [30]:
cnt_vectorizer.fit(train_text.iloc[:2])

c:\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x00000243B714C0D0>)

문장에서 나온 단어들을 다음과 같음.

In [36]:
cnt_vectorizer.vocabulary_

# am이 0으로 mappint된다는 의미.

{'am': 0,
 'only': 8,
 'searching': 12,
 'for': 4,
 'good': 5,
 'dual': 2,
 'sim': 14,
 'mobile': 7,
 'pa': 9,
 'excellent': 3,
 'ill': 6,
 'see': 13,
 'what': 15,
 'rileys': 11,
 'plans': 10,
 'are': 1}

In [32]:
vocab = sorted(cnt_vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab = list(map(lambda x: x[0], vocab))
vocab

['am',
 'are',
 'dual',
 'excellent',
 'for',
 'good',
 'ill',
 'mobile',
 'only',
 'pa',
 'plans',
 'rileys',
 'searching',
 'see',
 'sim',
 'what']

- 전에 fitting한 count_vectorize로 변환시킬 수 있음.

In [33]:
sample_cnt_vector = cnt_vectorizer.transform(train_text.iloc[:2]).toarray()
sample_cnt_vector

array([[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1]], dtype=int64)

In [34]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [35]:
pd.DataFrame(sample_cnt_vector, columns=vocab)

,am,are,dual,excellent,for,good,ill,mobile,only,pa,plans,rileys,searching,see,sim,what
0,1,0,1,0,1,1,0,1,1,1,0,0,1,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,1


#### 2.2.1 학습

- 이제 모든 데이터에 대해 진행.

In [37]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text)

c:\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x00000243B714C0D0>)

전체 단어는 7908개 존재.

In [38]:
len(cnt_vectorizer.vocabulary_)

7908

#### 2.2.2 예측

In [39]:
train_matrix = cnt_vectorizer.transform(train_text)
test_matrix = cnt_vectorizer.transform(test_text)

- 만약 존재하지 않는 단어가 들어올 경우,
    - CountVectorize는 학습한 단어장에 존재하지 않는 단어가 들어오게 될 경우 무시.

In [40]:
cnt_vectorizer.transform(['notavailablewordforcnt']).toarray().sum()

0

## 3. Naive Bayes

- 분류를 위한 Naive Bayes 모델은 BernoulliNB를 사용.

In [41]:
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()

### 3.1 학습

In [42]:
# train_matrix: count_vectorize로 변환시켜줬음.

naive_bayes.fit(train_matrix, train_label)

BernoulliNB()

### 3.2 예측

In [43]:
train_pred = naive_bayes.predict(train_matrix)
test_pred = naive_bayes.predict(test_matrix)

### 3.3 평가

In [44]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

In [45]:
print(f"Train Accuracy is {train_acc:.4f}")
print(f"Test Accuracy is {test_acc:.4f}")

Train Accuracy is 0.9854
Test Accuracy is 0.9767
